In [ ]:
%matplotlib inline

import io, os, sys, types, datetime, pickle, warnings

warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float, filters, morphology, transform

from sklearn import linear_model
from sklearn import metrics

In [ ]:
""" ============== path settings =============="""

In [ ]:
module_path = os.path.join(os.path.dirname(os.getcwd()), 'python_cluster', 'functions')

In [ ]:
module_path

In [ ]:
# env = 'Windows'
# if(env == 'Windows'):
#     module_path = 'W:\\2019_09_NSP_Extension\\code\\NSP_codes\\python_cluster\\helper_functions'
# elif(env == 'Mac'):
#     module_path = '/Users/lily/Lily/Academic/AW_Lab/code/python_cluster/helper_functions'

In [ ]:
# Mac, Figure_Output1, Data_Output1, Ctrl_22hrs_Gal80_s3r101_annotation.csv, 0, 1, 15, 5, 20, 5, 1

In [ ]:
### import custom functions
if module_path not in sys.path:
    sys.path.append(module_path)
import settings as settings
import helper as my_help
import intensity_calculation as my_int
import parse_bundle as my_pb
import plotting4 as my_plot

In [ ]:
paths = settings.paths
matching_info = settings.matching_info
analysis_params_general = settings.analysis_params_general

In [ ]:
matching_info

In [ ]:
paths.code_path

In [ ]:
"""============== main =============="""

In [ ]:
sns.set_style("dark")

In [ ]:
paths.annot_path

In [ ]:
"""Load data"""
### load summaries
summary_df = pd.read_csv(paths.annot_path)
image_list = summary_df.loc[:,'Image_Name'].unique()
ROI_list = summary_df.loc[:,'ROI_Name'].unique()

In [ ]:
image_list, ROI_list

In [ ]:
i_image = 0

In [ ]:
### load other data
image_name = image_list[i_image]
roi_name = ROI_list[i_image]
roi_df = pd.read_csv(os.path.join(paths.roi_path, roi_name))
roi_df.rename(columns = {' ':'No'}, inplace = True)
annot_df = summary_df.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)
m2p_ratio = (summary_df.iloc[0]['imgX_pixel']/summary_df.iloc[0]['imgX_um'], summary_df.iloc[0]['imgY_pixel']/summary_df.iloc[0]['imgY_um'])

In [ ]:
""" Process annotation info"""
is_extended_target_list = False
annotation_type = annot_df.loc[0,'Annotation_type']
if(annotation_type == 1):
    bundles_df = my_pb.get_bundles_info_v1(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
if(annotation_type == 2):
    bundles_df = my_pb.get_bundles_info_v2(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
if(annotation_type == 3):
    bundles_df = my_pb.get_bundles_info_v3(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
annot_bundles_df = bundles_df.dropna(axis=0, how='any', inplace = False)
annot_bundles_df.sort_index(inplace = True)

In [ ]:
annot_bundles_df['coord_Z_Center']

In [ ]:
"""load imaging data"""

In [ ]:
%time image = img_as_float(skiIo.imread(os.path.join(paths.image_path, image_name)))
image_shape = (image.shape[0], image.shape[1], image.shape[2])
image_info = [image_name, image_shape, m2p_ratio]

In [ ]:
""" Process images """
### number of channels
nChannels = min(image.shape)# number of channels of original image
is_process_channels = True
if not is_process_channels:
    num_norm_channels = len(matching_info.channel_mapping_checking.keys()) # number of channels of normalized image
else:
    num_norm_channels = len(matching_info.channel_cmap.keys())
    
if(nChannels == 2):
    print("2 channels!")
    ### normalize channels
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,0,:,:].dtype, order='C')
    thr = np.zeros((2))
    
    # RFP_norm
    %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,0,:,:], in_range = 'image', out_range='dtype')
    # GFP_norm
    %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,1,:,:], in_range = 'image', out_range='dtype')    
    
    del image
        
elif(nChannels == 4):
    print("4 channels!")
    ### normalize channels
    if not is_process_channels:
        image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,:,:,1].dtype, order='C')
        # RFP_norm
        %time image_norm[:,:,:,0] = image[:,:,:,0]
        # GFP_norm
        %time image_norm[:,:,:,1] = image[:,:,:,1]
        # 24b10
        %time image_norm[:,:,:,2] = image[:,:,:,2]
        # FasII
        %time image_norm[:,:,:,3] = image[:,:,:,3]

        del image
    else:
        thr = np.zeros((3))
        image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,:,:,1].dtype, order='C')
        # RFP_norm
        %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,:,:,0], in_range = 'image', out_range='dtype')
        # GFP_norm
        %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,:,:,1], in_range = 'image', out_range='dtype')
        # FasII norm
        %time image_norm[:,:,:,7] = exposure.rescale_intensity(image[:,:,:,3], in_range = 'image', out_range='dtype')
        
        %time thr[0] = filters.threshold_isodata(image_norm[:,:,:,1])
        %time thr[1] = filters.threshold_mean(image_norm[:,:,:,1])
        %time thr[2] = filters.threshold_otsu(image_norm[:,:,:,7])

        %time gfp = transform.match_histograms(image_norm[:,:,:,1], image_norm[:,:,:,0])

        print("R3/R4 v1")
        r3_img = image_norm[:,:,:,0] - gfp
        r3_img[r3_img<0] = 0
        image_norm[:,:,:,2] = exposure.rescale_intensity(r3_img, in_range = 'image', out_range='dtype')
        r4_img = image_norm[:,:,:,0] * gfp
        image_norm[:,:,:,3] = exposure.rescale_intensity(r4_img, in_range = 'image', out_range='dtype')

        print("R3/R4 v2")
        gfp_thr = morphology.binary_opening((image_norm[:,:,:,1]>thr[0])*1)
        image_norm[:,:,:,4] = exposure.rescale_intensity(image_norm[:,:,:,0] * (1-gfp_thr), in_range = 'image', out_range='dtype')
        image_norm[:,:,:,5] = exposure.rescale_intensity(morphology.closing(image_norm[:,:,:,1]*((image_norm[:,:,:,1]>((thr[0] + thr[1])/2))*1)))

        print("R3 v3")
        my_help.print_to_log("R3 v3")
        r3_img = image_norm[:,:,:,0] - gfp*settings.analysis_params_general.scale_factor
        r3_img[r3_img<0] = 0
        image_norm[:,:,:,6] = exposure.rescale_intensity(r3_img, in_range = 'image', out_range='dtype')
        
        print("FasII intersect")
        fasii_thr = morphology.binary_opening((image_norm[:,:,:,7]>thr[2])*1)
        image_norm[:,:,:,8] = exposure.rescale_intensity(image_norm[:,:,:,0] * fasii_thr, in_range = 'image', out_range='dtype')
        image_norm[:,:,:,9] = exposure.rescale_intensity(image_norm[:,:,:,1] * fasii_thr, in_range = 'image', out_range='dtype')

In [ ]:
"""example plot"""

In [ ]:
bundle_no = 1
plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
my_plot.plot_individual_bundles(bundle_no, bundles_df, image_norm, m2p_ratio[0], m2p_ratio[1], num_subplots = image_norm.shape[-1],
                                is_plot_r3_line = False, is_plot_r4_line = False, is_plot_r4 = False, is_label_off = True)
plt.show()

In [ ]:
"""plot all"""

In [ ]:
fasii_thr

In [ ]:
for ind, bundle_no in enumerate(annot_bundles_df.index):
    plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
    my_plot.plot_individual_bundles(bundle_no, bundles_df, image_norm, m2p_ratio[0], m2p_ratio[1], 
                                    num_subplots = image_norm.shape[-1], is_plot_r3_line = False, is_plot_r4_line = False, is_plot_r4 = False, is_label_off = True)
    plt.show()

In [ ]:
"""analyze images"""

In [ ]:
import time

In [ ]:
### parameters
analysis_params_general = settings.analysis_params_general
matching_info = settings.matching_info

### initialization
print('-----' + image_name + '------')
my_help.print_to_log('-----' + image_name + '------')
matrix_y = analysis_params_general.num_angle_section + 2 * analysis_params_general.num_outside_angle + 1
matrix_x = analysis_params_general.num_x_section + 1
matrix_z = analysis_params_general.z_offset * 2 + 1
num_norm_channels = image_norm.shape[-1]

intensity_matrix = np.zeros((len(annot_bundles_df.index), num_norm_channels, matrix_y, matrix_x, matrix_z))
intensity_matrix = intensity_matrix - 100

params = [];
rel_points = np.zeros((len(annot_bundles_df.index), 9))

### thresholds
thr_otsu = np.zeros((num_norm_channels))
thr_li = np.zeros((num_norm_channels))
thr_isodata = np.zeros((num_norm_channels))
time_start = time.time()
for channel_no in range(num_norm_channels):
    thr_otsu[channel_no] = filters.threshold_otsu(image_norm[:,:,:,channel_no])
    thr_li[channel_no] = filters.threshold_li(image_norm[:,:,:,channel_no])
    thr_isodata[channel_no] = filters.threshold_isodata(image_norm[:,:,:,channel_no])
time_end = time.time()
time_dur = time_end - time_start
my_help.print_to_log("total time: " + str(time_dur))

### process
for ind, bundle_no in enumerate(annot_bundles_df.index):
# ind = 0
# bundle_no = 1
    print("Bundle No: " + str(bundle_no))
    my_help.print_to_log("Bundle No: " + str(bundle_no))

    ### targets info
    ind_targets, coord_targets = my_help.get_target_coords(bundle_no, bundles_df, matching_info.index_to_target_id)
    coord_center = my_help.get_bundle_center(bundle_no, bundles_df)
    coord_r4s = my_help.get_rx_coords(bundle_no, bundles_df, ind_targets, 4)
    coord_r3s = my_help.get_rx_coords(bundle_no, bundles_df, ind_targets, 3)
    coord_rcells = np.concatenate((coord_r4s, coord_r3s))

    ### slice info
    slice_zero_point = coord_targets[matching_info.target_id_to_index[7],:] # T3'
    slice_one_point = coord_targets[matching_info.target_id_to_index[3],:] # T3

    length_one_point = coord_targets[matching_info.target_id_to_index[4],:]

    center_points = [coord_targets[0,:], coord_center[0,:]]

    r_cell_nos = [4,4]


    ### get slicing params and calculate matrix
    center_type = settings.analysis_params_general.center_type
    slice_type = settings.analysis_params_general.slice_type

    bundle_params = [
        bundle_no, 
        ind_targets, 
        coord_targets, 
        coord_center, 
        slice_zero_point, 
        slice_one_point, 
        length_one_point, 
        center_points[analysis_params_general.center_type], 
        r_cell_nos[analysis_params_general.center_type]
    ]
    if(slice_type == 0):
        pp_i, rel_points_i = my_int.get_slice_params_v1(bundles_df, bundle_params, is_print = False, is_plot = False)
    elif(slice_type == 1):
        pp_i, rel_points_i = my_int.get_slice_params_v3(bundles_df, bundle_params, is_print = False, is_plot = False)
    params.append(pp_i)
    rel_points[ind, :] = rel_points_i

    # calculate matrix
    time_start = time.time()
    for channel_no in range(num_norm_channels):
        my_help.print_to_log("Channle No: " + str(channel_no))
        intensity_matrix[ind, channel_no,:,:,:] = my_int.get_intensity_matrix_new(pp_i, image_norm[:,:,:,channel_no])
        # intensity_matrix[ind, channel_no,:,:,:] = np.random.randn(intensity_matrix[ind, channel_no,:,:,:].shape[0], intensity_matrix[ind, channel_no,:,:,:].shape[1], intensity_matrix[ind, channel_no,:,:,:].shape[2])
    time_end = time.time()
    time_dur = time_end - time_start
    my_help.print_to_log("total time: " + str(time_dur))

In [ ]:
paths = settings.paths
analysis_params_general = settings.analysis_params_general
matching_info = settings.matching_info
num_norm_channels = image_norm.shape[-1]
img_name = image_info[0]

for ind, bundle_no in enumerate(annot_bundles_df.index):
    print("Bundle No: ", bundle_no)
    my_help.print_to_log("Bundle No: " + str(bundle_no))

    category_id = annot_bundles_df.iloc[0]['CategoryID']
    sample_id = annot_bundles_df.iloc[0]['SampleID']
    region_id = annot_bundles_df.iloc[0]['RegionID']



    ### targets info
    ind_targets, coord_targets = my_help.get_target_coords(bundle_no, bundles_df, matching_info.index_to_target_id)
    coord_center = my_help.get_bundle_center(bundle_no, bundles_df)
    coord_r4s = my_help.get_rx_coords(bundle_no, bundles_df, ind_targets, 4)
    coord_r3s = my_help.get_rx_coords(bundle_no, bundles_df, ind_targets, 3)
    coord_rs = np.concatenate((coord_r4s, coord_r3s))

    ### parameters
    pp_i = params[ind]
    rel_points_i = rel_points[ind, :]

    matrix = my_help.delete_zero_columns(intensity_matrix[ind, :, :, :, :], -100, 3)
    if(len(matrix.flatten()) > 0):

        ## heat map
        plt.ioff()
        ori_x = np.round(np.linspace(0, analysis_params_general.radius_expanse_ratio[analysis_params_general.center_type], matrix.shape[2]), 2)
        tick_params = [2, 1, ori_x, 21] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
        for thr_function_ids in [0, 1, 2, 3]: # different thresholding methods
            thrs = np.zeros((num_norm_channels))
            if(thr_function_ids == 0):
                thrs = np.zeros((num_norm_channels))
            elif(thr_function_ids == 1):
                thrs = thr_otsu
            elif(thr_function_ids == 2):
                thrs = thr_li
            elif(thr_function_ids == 3):
                thrs = thr_isodata

            fig_name = f'{category_id}_s{sample_id}r{region_id}_bundle_no_{bundle_no}_{thr_function_ids}.png'
            fig_params = [pp_i, img_name, fig_name]
            plot_options = [thrs, thr_function_ids, num_norm_channels]
            fig = my_plot.plot_bundle_vs_matrix_all(bundle_no, bundles_df, image_norm, matrix, fig_params, tick_params, plot_options, is_label_off = True, is_save = True, is_true_x_tick = True, is_ori_tick = False)
            plt.close(fig)

        ## polar plot
        fig_params = [pp_i, img_name]
        # plot_options = [True, True] # isLabelOff, isSave
        for channel_no in range(num_norm_channels):
            fig = my_plot.plot_polar(bundle_no, bundles_df, image_norm, channel_no, matrix, fig_params, rel_points_i, is_label_off = True, is_save = True)
            plt.close(fig)

    else:
        print("error! No intensity matrix calculated!")
